In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import numpy as np
from scipy import spatial
from skimage import measure
from skimage import transform
import tensorflow as tf

from google.protobuf import text_format
from tensorflow.python.platform import app
from delf import aggregation_config_pb2
from delf import datum_io
from delf import feature_aggregation_similarity
from delf import feature_io



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
# Aliases for aggregation types.
_VLAD = aggregation_config_pb2.AggregationConfig.VLAD
_ASMK = aggregation_config_pb2.AggregationConfig.ASMK
_ASMK_STAR = aggregation_config_pb2.AggregationConfig.ASMK_STAR

# Extensions.
_DELF_EXTENSION = '.delf'
_VLAD_EXTENSION_SUFFIX = 'vlad'
_ASMK_EXTENSION_SUFFIX = 'asmk'
_ASMK_STAR_EXTENSION_SUFFIX = 'asmk_star'

# Precision-recall ranks to use in metric computation.
_PR_RANKS = (1, 5, 10)

# Pace to log.
_STATUS_CHECK_LOAD_ITERATIONS = 50
_STATUS_CHECK_GV_ITERATIONS = 10

# Output file names.
_METRICS_FILENAME = 'metrics.txt'

# Re-ranking / geometric verification parameters.
_NUM_TO_RERANK = 100
_FEATURE_DISTANCE_THRESHOLD = 0.9
_NUM_RANSAC_TRIALS = 1000
_MIN_RANSAC_SAMPLES = 3
_RANSAC_RESIDUAL_THRESHOLD = 10

In [3]:


def _ReadAggregatedDescriptors(input_dir, image_list, config):
    """Reads aggregated descriptors.

    Args:
        input_dir: Directory where aggregated descriptors are located.
        image_list: List of image names for which to load descriptors.
        config: AggregationConfig used for images.

    Returns:
        aggregated_descriptors: List containing #images items, each a 1D NumPy
            array.
        visual_words: If using VLAD aggregation, returns an empty list. Otherwise,
            returns a list containing #images items, each a 1D NumPy array.
    """
    # Compose extension of aggregated descriptors.
    extension = '.'
    if config.use_regional_aggregation:
        extension += 'r'
    if config.aggregation_type == _VLAD:
        extension += _VLAD_EXTENSION_SUFFIX
    elif config.aggregation_type == _ASMK:
        extension += _ASMK_EXTENSION_SUFFIX
    elif config.aggregation_type == _ASMK_STAR:
        extension += _ASMK_STAR_EXTENSION_SUFFIX
    else:
        raise ValueError('Invalid aggregation type: %d' % config.aggregation_type)

    num_images = len(image_list)
    aggregated_descriptors = []
    visual_words = []
    print('Starting to collect descriptors for %d images...' % num_images)
    start = time.clock()
    for i in range(num_images):
        if i > 0 and i % _STATUS_CHECK_LOAD_ITERATIONS == 0:
            elapsed = (time.clock() - start)
            print('Reading descriptors for image %d out of %d, last %d '
                        'images took %f seconds' %
                        (i, num_images, _STATUS_CHECK_LOAD_ITERATIONS, elapsed))
            start = time.clock()

        descriptors_filename = image_list[i] + extension
        descriptors_fullpath = os.path.join(input_dir, descriptors_filename)
        if config.aggregation_type == _VLAD:
            aggregated_descriptors.append(datum_io.ReadFromFile(descriptors_fullpath))
        else:
            d, v = datum_io.ReadPairFromFile(descriptors_fullpath)
            if config.aggregation_type == _ASMK_STAR:
                d = d.astype('uint8')

            aggregated_descriptors.append(d)
            visual_words.append(v)

    return aggregated_descriptors, visual_words


In [4]:

def _MatchFeatures(query_locations, query_descriptors, index_image_locations,
                                     index_image_descriptors):
    """Matches local features using geometric verification.

    First, finds putative local feature matches by matching `query_descriptors`
    against a KD-tree from the `index_image_descriptors`. Then, attempts to fit an
    affine transformation between the putative feature corresponces using their
    locations.

    Args:
        query_locations: Locations of local features for query image. NumPy array of
            shape [#query_features, 2].
        query_descriptors: Descriptors of local features for query image. NumPy
            array of shape [#query_features, depth].
        index_image_locations: Locations of local features for index image. NumPy
            array of shape [#index_image_features, 2].
        index_image_descriptors: Descriptors of local features for index image.
            NumPy array of shape [#index_image_features, depth].

    Returns:
        score: Number of inliers of match. If no match is found, returns 0.
    """
    num_features_query = query_locations.shape[0]
    num_features_index_image = index_image_locations.shape[0]
    if not num_features_query or not num_features_index_image:
        return 0

    # Find nearest-neighbor matches using a KD tree.
    index_image_tree = spatial.cKDTree(index_image_descriptors)
    _, indices = index_image_tree.query(
            query_descriptors, distance_upper_bound=_FEATURE_DISTANCE_THRESHOLD)

    # Select feature locations for putative matches.
    query_locations_to_use = np.array([
            query_locations[i,]
            for i in range(num_features_query)
            if indices[i] != num_features_index_image
    ])
    index_image_locations_to_use = np.array([
            index_image_locations[indices[i],]
            for i in range(num_features_query)
            if indices[i] != num_features_index_image
    ])

    # If there are no putative matches, early return 0.
    if not query_locations_to_use.shape[0]:
        return 0

    # Perform geometric verification using RANSAC.
    _, inliers = measure.ransac(
            (index_image_locations_to_use, query_locations_to_use),
            transform.AffineTransform,
            min_samples=_MIN_RANSAC_SAMPLES,
            residual_threshold=_RANSAC_RESIDUAL_THRESHOLD,
            max_trials=_NUM_RANSAC_TRIALS)
    if inliers is None:
        inliers = []

    return sum(inliers)


In [5]:


def _RerankByGeometricVerification(input_ranks, initial_scores, query_name,
                                                                     index_names, query_features_dir,
                                                                     index_features_dir, junk_ids):
    """Re-ranks retrieval results using geometric verification.

    Args:
        input_ranks: 1D NumPy array with indices of top-ranked index images, sorted
            from the most to the least similar.
        initial_scores: 1D NumPy array with initial similarity scores between query
            and index images. Entry i corresponds to score for image i.
        query_name: Name for query image (string).
        index_names: List of names for index images (strings).
        query_features_dir: Directory where query local feature file is located
            (string).
        index_features_dir: Directory where index local feature files are located
            (string).
        junk_ids: Set with indices of junk images which should not be considered
            during re-ranking.

    Returns:
        output_ranks: 1D NumPy array with index image indices, sorted from the most
            to the least similar according to the geometric verification and initial
            scores.

    Raises:
        ValueError: If `input_ranks`, `initial_scores` and `index_names` do not have
            the same number of entries.
    """
    num_index_images = len(index_names)
    if len(input_ranks) != num_index_images:
        raise ValueError('input_ranks and index_names have different number of '
                                         'elements: %d vs %d' %
                                         (len(input_ranks), len(index_names)))
    if len(initial_scores) != num_index_images:
        raise ValueError('initial_scores and index_names have different number of '
                                         'elements: %d vs %d' %
                                         (len(initial_scores), len(index_names)))

    # Filter out junk images from list that will be re-ranked.
    input_ranks_for_gv = []
    for ind in input_ranks:
        if ind not in junk_ids:
            input_ranks_for_gv.append(ind)
    num_to_rerank = min(_NUM_TO_RERANK, len(input_ranks_for_gv))

    # Load query image features.
    query_features_path = os.path.join(query_features_dir,
                                                                         query_name + _DELF_EXTENSION)
    query_locations, _, query_descriptors, _, _ = feature_io.ReadFromFile(
            query_features_path)

    # Initialize list containing number of inliers and initial similarity scores.
    inliers_and_initial_scores = []
    for i in range(num_index_images):
        inliers_and_initial_scores.append([0, initial_scores[i]])

    # Loop over top-ranked images and get results.
    print('Starting to re-rank')
    for i in range(num_to_rerank):
        if i > 0 and i % _STATUS_CHECK_GV_ITERATIONS == 0:
            print('Re-ranking: i = %d out of %d' % (i, num_to_rerank))

        index_image_id = input_ranks_for_gv[i]

        # Load index image features.
        index_image_features_path = os.path.join(
                index_features_dir, index_names[index_image_id] + _DELF_EXTENSION)
        (index_image_locations, _, index_image_descriptors, _,
         _) = feature_io.ReadFromFile(index_image_features_path)

        inliers_and_initial_scores[index_image_id][0] = _MatchFeatures(
                query_locations, query_descriptors, index_image_locations,
                index_image_descriptors)

    # Sort based on (inliers_score, initial_score).
    def _InliersInitialScoresSorting(k):
        """Helper function to sort list based on two entries.

        Args:
            k: Index into `inliers_and_initial_scores`.

        Returns:
            Tuple containing inlier score and initial score.
        """
        return (inliers_and_initial_scores[k][0], inliers_and_initial_scores[k][1])

    output_ranks = sorted(
            range(num_index_images), key=_InliersInitialScoresSorting, reverse=True)

    return output_ranks


In [6]:

def _SaveMetricsFile(mean_average_precision, mean_precisions, mean_recalls,
                                         pr_ranks, output_path):
    """Saves aggregated retrieval metrics to text file.

    Args:
        mean_average_precision: Dict mapping each dataset protocol to a float.
        mean_precisions: Dict mapping each dataset protocol to a NumPy array of
            floats with shape [len(pr_ranks)].
        mean_recalls: Dict mapping each dataset protocol to a NumPy array of floats
            with shape [len(pr_ranks)].
        pr_ranks: List of integers.
        output_path: Full file path.
    """
    with tf.gfile.GFile(output_path, 'w') as f:
        for k in sorted(mean_average_precision.keys()):
            f.write('{}\n    mAP={}\n    mP@k{} {}\n    mR@k{} {}\n'.format(
                    k, np.around(mean_average_precision[k] * 100, decimals=2),
                    np.array(pr_ranks), np.around(mean_precisions[k] * 100, decimals=2),
                    np.array(pr_ranks), np.around(mean_recalls[k] * 100, decimals=2)))


In [7]:

# Parse dataset to obtain query/index images, and ground-truth.
print('Parsing dataset...')
query_list = os.listdir("image_test")
index_list = os.listdir("image_index")
num_query_images = len(query_list)
num_index_images = len(index_list)

print('done! Found %d queries and %d index images' %
            (num_query_images, num_index_images))

# Parse AggregationConfig protos.
query_config = aggregation_config_pb2.AggregationConfig()
with tf.gfile.GFile('query_aggregation_config.pbtxt', 'r') as f:
    text_format.Merge(f.read(), query_config)
index_config = aggregation_config_pb2.AggregationConfig()
with tf.gfile.GFile('index_aggregation_config.pbtxt', 'r') as f:
    text_format.Merge(f.read(), index_config)

# Read aggregated descriptors.
query_aggregated_descriptors, query_visual_words = _ReadAggregatedDescriptors('output/agg/query', query_list, query_config)
index_aggregated_descriptors, index_visual_words = _ReadAggregatedDescriptors('output/agg/index', index_list, index_config)

# Create similarity computer.
similarity_computer = (feature_aggregation_similarity.SimilarityAggregatedRepresentation(index_config))

# Compute similarity between query and index images, potentially re-ranking
# with geometric verification.
ranks_before_gv = np.zeros([num_query_images, num_index_images],dtype='int32')

use_geometric_verification = True

#if cmd_args.use_geometric_verification:
medium_ranks_after_gv = np.zeros([num_query_images, num_index_images],dtype='int32')
hard_ranks_after_gv = np.zeros([num_query_images, num_index_images],dtype='int32')

for i in range(num_query_images):
    print('Performing retrieval with query %d (%s)...' % (i, query_list[i]))
    start = time.clock()

    # Compute similarity between aggregated descriptors.
    similarities = np.zeros([num_index_images])
    for j in range(num_index_images):
        similarities[j] = similarity_computer.ComputeSimilarity(
                query_aggregated_descriptors[i], index_aggregated_descriptors[j],
                query_visual_words[i], index_visual_words[j])

    ranks_before_gv[i] = np.argsort(-similarities)

    elapsed = (time.clock() - start)
    print('done! Retrieval for query %d took %f seconds' % (i, elapsed))

# Create output directory if necessary.
if not os.path.exists('output/result'):
    os.makedirs('output/result')
    


Parsing dataset...
done! Found 5879 queries and 7618 index images
Starting to collect descriptors for 5879 images...
Reading descriptors for image 50 out of 5879, last 50 images took 0.033358 seconds
Reading descriptors for image 100 out of 5879, last 50 images took 0.032734 seconds
Reading descriptors for image 150 out of 5879, last 50 images took 0.029864 seconds
Reading descriptors for image 200 out of 5879, last 50 images took 0.029226 seconds
Reading descriptors for image 250 out of 5879, last 50 images took 0.032188 seconds


C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:41: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Reading descriptors for image 300 out of 5879, last 50 images took 0.053723 seconds
Reading descriptors for image 350 out of 5879, last 50 images took 0.041246 seconds
Reading descriptors for image 400 out of 5879, last 50 images took 0.030981 seconds
Reading descriptors for image 450 out of 5879, last 50 images took 0.030775 seconds
Reading descriptors for image 500 out of 5879, last 50 images took 0.033449 seconds
Reading descriptors for image 550 out of 5879, last 50 images took 0.033631 seconds
Reading descriptors for image 600 out of 5879, last 50 images took 0.036138 seconds
Reading descriptors for image 650 out of 5879, last 50 images took 0.034950 seconds
Reading descriptors for image 700 out of 5879, last 50 images took 0.033986 seconds
Reading descriptors for image 750 out of 5879, last 50 images took 0.043170 seconds
Reading descriptors for image 800 out of 5879, last 50 images took 0.042361 seconds
Reading descriptors for image 850 out of 5879, last 50 images took 0.035582 

Reading descriptors for image 5300 out of 5879, last 50 images took 0.036944 seconds
Reading descriptors for image 5350 out of 5879, last 50 images took 0.033462 seconds
Reading descriptors for image 5400 out of 5879, last 50 images took 0.035289 seconds
Reading descriptors for image 5450 out of 5879, last 50 images took 0.034407 seconds
Reading descriptors for image 5500 out of 5879, last 50 images took 0.034529 seconds
Reading descriptors for image 5550 out of 5879, last 50 images took 0.034865 seconds
Reading descriptors for image 5600 out of 5879, last 50 images took 0.030615 seconds
Reading descriptors for image 5650 out of 5879, last 50 images took 0.034242 seconds
Reading descriptors for image 5700 out of 5879, last 50 images took 0.035585 seconds
Reading descriptors for image 5750 out of 5879, last 50 images took 0.028982 seconds
Reading descriptors for image 5800 out of 5879, last 50 images took 0.048698 seconds
Reading descriptors for image 5850 out of 5879, last 50 images to

Reading descriptors for image 4300 out of 7618, last 50 images took 0.062169 seconds
Reading descriptors for image 4350 out of 7618, last 50 images took 0.046258 seconds
Reading descriptors for image 4400 out of 7618, last 50 images took 0.041919 seconds
Reading descriptors for image 4450 out of 7618, last 50 images took 0.043023 seconds
Reading descriptors for image 4500 out of 7618, last 50 images took 0.053064 seconds
Reading descriptors for image 4550 out of 7618, last 50 images took 0.050692 seconds
Reading descriptors for image 4600 out of 7618, last 50 images took 0.045284 seconds
Reading descriptors for image 4650 out of 7618, last 50 images took 0.051853 seconds
Reading descriptors for image 4700 out of 7618, last 50 images took 0.047832 seconds
Reading descriptors for image 4750 out of 7618, last 50 images took 0.043356 seconds
Reading descriptors for image 4800 out of 7618, last 50 images took 0.046218 seconds
Reading descriptors for image 4850 out of 7618, last 50 images to

...

done! Retrieval for query 5626 took 3.899253 seconds
Performing retrieval with query 5627 (0c58b3b9ba39e072.jpg)...
done! Retrieval for query 5627 took 1.667814 seconds
Performing retrieval with query 5628 (0c58bdf56ada6eed.jpg)...
done! Retrieval for query 5628 took 6.836960 seconds
Performing retrieval with query 5629 (0c5918038557d996.jpg)...
done! Retrieval for query 5629 took 2.598284 seconds
Performing retrieval with query 5630 (0c5a5eccf6e30e71.jpg)...
done! Retrieval for query 5630 took 3.201395 seconds
Performing retrieval with query 5631 (0c5b59e360437ec0.jpg)...
done! Retrieval for query 5631 took 2.195855 seconds
Performing retrieval with query 5632 (0c5bb72088d85104.jpg)...
done! Retrieval for query 5632 took 2.477288 seconds
Performing retrieval with query 5633 (0c5be6372dec2a4a.jpg)...
done! Retrieval for query 5633 took 3.666951 seconds
Performing retrieval with query 5634 (0c5bf0be6cfd69dd.jpg)...
done! Retrieval for query 5634 took 4.756644 seconds
Performing retrieva

done! Retrieval for query 5697 took 3.695366 seconds
Performing retrieval with query 5698 (0c7a6a057200712b.jpg)...
done! Retrieval for query 5698 took 3.538347 seconds
Performing retrieval with query 5699 (0c7a6bcd4246c434.jpg)...
done! Retrieval for query 5699 took 4.010260 seconds
Performing retrieval with query 5700 (0c7bbced04352dc6.jpg)...
done! Retrieval for query 5700 took 5.591448 seconds
Performing retrieval with query 5701 (0c7bcf1228db6e79.jpg)...
done! Retrieval for query 5701 took 3.143050 seconds
Performing retrieval with query 5702 (0c7d0b292a7f1501.jpg)...
done! Retrieval for query 5702 took 3.037090 seconds
Performing retrieval with query 5703 (0c7f27e98d6534cd.jpg)...
done! Retrieval for query 5703 took 1.703648 seconds
Performing retrieval with query 5704 (0c7ffb4b77be7abd.jpg)...
done! Retrieval for query 5704 took 2.005751 seconds
Performing retrieval with query 5705 (0c80088e8af40a21.jpg)...
done! Retrieval for query 5705 took 1.369752 seconds
Performing retrieva

done! Retrieval for query 5768 took 5.725791 seconds
Performing retrieval with query 5769 (0ca41cdbefb99cd4.jpg)...
done! Retrieval for query 5769 took 3.931066 seconds
Performing retrieval with query 5770 (0ca42a4ef19d6eec.jpg)...
done! Retrieval for query 5770 took 3.137566 seconds
Performing retrieval with query 5771 (0ca46c75742a4d36.jpg)...
done! Retrieval for query 5771 took 2.832366 seconds
Performing retrieval with query 5772 (0ca48dfa09f3ffc9.jpg)...
done! Retrieval for query 5772 took 3.517587 seconds
Performing retrieval with query 5773 (0ca4e2f2bb0394ad.jpg)...
done! Retrieval for query 5773 took 3.925881 seconds
Performing retrieval with query 5774 (0ca68d1b541133d9.jpg)...
done! Retrieval for query 5774 took 5.497515 seconds
Performing retrieval with query 5775 (0ca6d2b8ca28545c.jpg)...
done! Retrieval for query 5775 took 3.999265 seconds
Performing retrieval with query 5776 (0ca7937523f641b0.jpg)...
done! Retrieval for query 5776 took 3.688291 seconds
Performing retrieva

done! Retrieval for query 5839 took 2.317236 seconds
Performing retrieval with query 5840 (0cc4c3afb6c257f9.jpg)...
done! Retrieval for query 5840 took 2.222235 seconds
Performing retrieval with query 5841 (0cc55eb2e8f76fa4.jpg)...
done! Retrieval for query 5841 took 3.350569 seconds
Performing retrieval with query 5842 (0cc59dc4a6edae1a.jpg)...
done! Retrieval for query 5842 took 3.497686 seconds
Performing retrieval with query 5843 (0cc5a7b40b8f3705.jpg)...
done! Retrieval for query 5843 took 4.855452 seconds
Performing retrieval with query 5844 (0cc649860efe093d.jpg)...
done! Retrieval for query 5844 took 2.012840 seconds
Performing retrieval with query 5845 (0cc6e08cfa3a0eb3.jpg)...
done! Retrieval for query 5845 took 4.443724 seconds
Performing retrieval with query 5846 (0cc77de7ae2ff9eb.jpg)...
done! Retrieval for query 5846 took 3.848639 seconds
Performing retrieval with query 5847 (0cc9112443a51b37.jpg)...
done! Retrieval for query 5847 took 1.840282 seconds
Performing retrieva

ValueError: Unsupported array type: int32

In [19]:
test_list = os.listdir("image_test")
index_list = os.listdir("image_index")

In [24]:
import pandas as pd

In [26]:
ranks = pd.DataFrame(ranks_before_gv)
test = pd.DataFrame(test_list)
index = pd.DataFrame(index_list)

In [108]:
ranks_100 = ranks.iloc[:,:100]
new_ranks = ranks_100.apply(lambda x: x.apply(lambda y: index_list[int(y)].replace('.jpg','')), axis = 1)
test_name = test[0].apply(lambda x: x.replace('.jpg',''))
result = pd.concat([test_name, new_ranks.apply(' '.join, axis = 1) ], axis = 1)
result.columns = ['id','images']
result.head(10)

,id,images
0,00016575233bc956,003797f78a87294b 01abf2e0148d706c 008ccc3c907c...
1,0001aadbcd8cb923,00b21c1fb57131a5 024de696a16e7e54 00d884317221...
2,0002c06b2440a5f9,01f8834279b8a63d 01f14f4b999a6a4b 011008d738e2...
3,0002eb1ee5a5a6b2,028a6fd249ac3d4c 00988a898305be61 01f8d10bc121...
4,000594dad986513e,00a6bcd7572ae6c8 006cd499a40c29d0 0106ecf5a164...
5,0006b7eb42bc7ee2,0105b32d1096eccf 0190bf4d897976bb 0022d66679a9...
6,00084cdf8f600d00,012dd1e17c793145 01f3299c92b52bb8 00511a08ee52...
7,0008722c48c52fda,026f6ef85aaf63e3 00768f2fe28783d5 01139da54995...
8,00090b69457cfbd3,010575a6f9499b80 013a0d1feae65df8 013466a67c60...
9,0009ffd36a94a88f,011e23cef46f0aa7 010d0ae1b88bf853 01d6055dd99d...


In [110]:
sample_submission = pd.read_csv("submission_sample.csv")
result_full = pd.merge(sample_submission, result, left_on='id', right_on='id', how='left' )
result_full.columns = ['id','images']

result_full.to_csv('submission_2015_05_29_2.csv', index=False)